In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
url = "https://jupiter.money/contact/"
headers = {"User-Agent": "Mozilla/5.0"}

In [ ]:

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
faq_section = soup.find('ul', {'data-controller': 'faq'})
faq_items = faq_section.find_all('li')


In [ ]:
faq_data = []
for item in faq_items:
    question = item.find('span').get_text(strip=True) if item.find('span') else ''
    answer = item.find('p').get_text(strip=True) if item.find('p') else ''
    if question and answer:
        faq_data.append({'question': question, 'answer': answer})


In [ ]:
# Save as CSV
df = pd.DataFrame(faq_data)
df.to_csv('faq_raw.csv', index=False)

In [ ]:
import re

In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [ ]:
df['question_cleaned'] = df['question'].str.lower().str.replace(r'[^\w\s]', '', regex=True)


In [ ]:
df = df.drop_duplicates(subset='question_cleaned')


In [ ]:
def categorize(question):
    if any(keyword in question.lower() for keyword in ['kyc', 'identity']):
        return 'KYC'
    elif any(keyword in question.lower() for keyword in ['reward', 'point']):
        return 'Rewards'
    elif any(keyword in question.lower() for keyword in ['payment', 'upi', 'transfer']):
        return 'Payments'
    elif any(keyword in question.lower() for keyword in ['limit', 'balance']):
        return 'Limits'
    else:
        return 'General'

df['category'] = df['question'].apply(categorize)


In [ ]:
!pip install transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prithivida/parrot_paraphraser_on_T5")
model = AutoModelForSeq2SeqLM.from_pretrained("prithivida/parrot_paraphraser_on_T5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def paraphrase_answer(answer, num_return_sequences=1):
    text = f"paraphrase: {answer} </s>"
    encoding = tokenizer.encode_plus(text, padding='longest', return_tensors="pt")
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=256,
        do_sample=True,
        top_k=120,
        top_p=0.95,
        early_stopping=True,
        num_return_sequences=num_return_sequences
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
# Example usage
original = "You can update your KYC details from the app under Profile > KYC."
print(paraphrase_answer(original))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


['You can update your KYC details from the app under Profile > KYC.']


In [ ]:
df['rephrased_answer'] = df['answer'].apply(lambda x: paraphrase_answer(x)[0])


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

In [ ]:
!pip install sentence-transformers faiss-cpu

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # small, fast, good quality

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
faq_questions = df['question'].tolist()
faq_embeddings = model.encode(faq_questions, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import faiss

In [ ]:
dimension = faq_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(faq_embeddings))

In [ ]:
def find_similar_faqs(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = []
    for idx in indices[0]:
        question = df.iloc[idx]['question']
        answer = df.iloc[idx]['rephrased_answer']
        results.append((question, answer))
    return results

In [ ]:
query = "Where do I change my KYC details?"
for q, a in find_similar_faqs(query):
    print(f"Matched FAQ: {q}\nAnswer: {a}\n")


Matched FAQ: How can I open a Savings
            account?
Answer: To open a free savings or salary bank account on Jupiter - powered by Federal Bank - in 3 minutes simply install the Jupiter App.Follow the on-screen instructions to create your account.

Matched FAQ: How can I set a PIN for my
            Debit Card?
Answer: You can set your debit card PIN by tapping on ‘PIN’ in the card tab of the Jupiter app.

Matched FAQ: How can I get a Federal Bank
            passbook?
Answer: You can request a Passbook by visiting your nearest Federal Bank branch. Please note that the Passbook is chargeable and is available only to verified account holders.



In [ ]:
def find_best_faq_match(query, threshold=0.4):  # adjust threshold as needed
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), 1)

    best_distance = distances[0][0]
    best_index = indices[0][0]

    if best_distance < threshold:  # closer means more similar
        question = df.iloc[best_index]['question']
        answer = df.iloc[best_index]['rephrased_answer']
        return answer
    else:
        return "I'm not sure about that. Could you rephrase or ask something else?"


In [ ]:
print(find_best_faq_match("Can I track my pizza order?"))
# Output: I'm not sure about that. Could you rephrase or ask something else?

print(find_best_faq_match("How do I complete my KYC?"))
# Output: <KYC-related answer>


I'm not sure about that. Could you rephrase or ask something else?
I'm not sure about that. Could you rephrase or ask something else?


In [ ]:
def chat_with_bot():
    print("🤖 Hi! Ask me anything about Jupiter's services. Type 'exit' to stop.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("🤖 Bye! Have a great day.")
            break

        response = find_best_faq_match(user_input)
        print(f"🤖 {response}\n")


In [ ]:
chat_with_bot()

🤖 Hi! Ask me anything about Jupiter's services. Type 'exit' to stop.

You: What is jupiter money
🤖 Jupiter is the 1-app for everything money that lets you spend and save money, track expenses, pay bills and invest money in direct mutual funds. It enables you to make smart money decisions every day using simple intuitive and personalized money management tools

You: Is Jupiter a bank?
🤖 Jupiter is not a bank and does not hold or claim to hold a banking license. The Savings Account and the VISA Debit Card are provided by Federal Bank and follow all security standards as applicable. All funds in your account are insured up to the approved limit by DICGC.

You: exit
🤖 Bye! Have a great day.


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
import streamlit as st

In [ ]:
st.title("🧠 Jupiter FAQ Bot")
st.write("Ask me anything about Jupiter's services!")

2025-06-22 12:06:05.404 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:05.840 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-22 12:06:05.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:05.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:05.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:05.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:05.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
user_input = st.text_input("Your Question")

2025-06-22 12:06:20.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.912 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.917 Session state does not function when running a script without `streamlit run`
2025-06-22 12:06:20.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.925 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 12:06:20.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
if user_input:
    response = find_best_faq_match(user_input)
    st.markdown(f"**🤖 Answer:** {response}")

In [ ]:
#

In [ ]:
chat_with_bot()

🤖 Hi! Ask me anything about Jupiter's services. Type 'exit' to stop.

You: hey what is jupiter used for
🤖 I'm not sure about that. Could you rephrase or ask something else?

You: what is jupiter
🤖 I'm not sure about that. Could you rephrase or ask something else?

You:  What is jupiter money
🤖 Jupiter is the 1-app for everything money that lets you spend and save money, track expenses, pay bills and invest money in direct mutual funds. It enables you to make smart money decisions every day using simple intuitive and personalized money management tools

You: exit
🤖 Bye! Have a great day.
